# RareSkills Zero Knowledge Week 1


## Modular arithmetic essentials
### Benchmarks
Here is how you do modular arithmetic in python. Run the two cells below and note how different their execution times are. You should use `pow` instead of doing modular arithmetic with the elementary operators.

In [18]:
import time

s = time.time()
ans = 1002583 ** 939001 % 2003951
t = time.time()

print("calculating {} took {:.2f} seconds".format(ans, t - s))

calculating 1590868 took 3.59 seconds


In [19]:
s = time.time()
ans = pow(1002583,939001,2003951)
t = time.time()

print("calculating {} operation took {:.2f} seconds".format(ans, t - s))

calculating 1590868 operation took 0.00 seconds


### Encrypting a number

If we start with a generator `g` and raise it to a power `s` mod `n`, then in general we cannot determine `s` even if `g` and `n` are known. This is the discrete logarithm problem.

We can say we "encrypt" `s` using the scheme

$$ E(s) = g^{s} \pmod n $$

But this means we need to choose `g` and `n` carefully. If `g` is 3, `s` is 2, and `n` is a large number, we can intelligently guess what `s` is in this case. We want to pick `g` and `n` such that all members "wrap around." We won't concern ourselves with this here, but you should be aware of it. The other issue is not picking a sufficiently large `n`. 

### Computing the discrete logarithm
If we do `s_p = g^s mod n`, and make `g` and `n` public, then it is infeasible to compute `s`. But this is only true if `n` is sufficiently large.

In the following example, we provide `s_p`, `g`, and `n`. Use python to bruteforce the solution to find `s`.

$$ g^{a*c} \pmod p = g^{a}^c \pmod p$$

In [20]:
n = 9551
g = 5
encrypted_number = 5666

<font color='red'>**Assignment 1: Use python to bruteforce the solution to find `s`**</font>

In [21]:
# Your Solution and Answer Here
n = 9551
g = 5
s_p = 5666

s = 0
result = 1

while result != s_p:
    s += 1
    result = (g ** s) % n

print("The value of s is:", s)

student_solution = s

The value of s is: 2531


In [22]:
assert pow(g, student_solution, n) == s_p
print("student_solution is {}".format(student_solution))

student_solution is 2531


### Computing the discrete logarithm the smart way
The smart way to do optimal algorithms is to [copy and paste from stackoverflow](https://stackoverflow.com/a/58665206/2079806). You do not have to understand how the library works

<font color='red'>**Assignment 2: Break the cryptography below and obtain the secret number**</font>

In [23]:
n = 1000004119
g = 5
encrypted_number = 767805982

In [24]:
# Your Solution and Answer Here
from sympy.ntheory import discrete_log
# discrete_log(p, input, base) -> log_g (s_p) = s mod(p)

discrete_log(n, 767805982, g)

student_solution = discrete_log(n, 767805982, g)

In [25]:
assert pow(g, student_solution, n) == encrypted_number
print("student_solution is {}".format(student_solution))

student_solution is 420


## Zero Knowledge Addition
The following property is very important. It lets us verify the addition of numbers without knowing the numbers.

$$ g^{a}g^{b} = g^{a + b} \pmod n $$

Try a few different values of `a` and `b` yourself to see it in action

In [26]:
# Student Inputs
a = 2
b = 3

print(pow(g, a, n) * pow(g, b, n) == pow(g, a + b, n))

True


Wait what?! That's supposed to be true?

<font color='red'>**Assignment 3: The code above has a bug. What is the bug?**</font>

In [27]:
# In the above code, you're calculating g**a (mod n) and g**b (mod n) separately and then multiplying them.
# The multiplication is not done under mod n. its just regular multiplication. 
# Therefore, not the same as g**(a+b) mod n

# Your corrected code here
result = pow(g, a + b, n) == (pow(g, a, n) * pow(g, b, n) % n)

print(result)

True


## Zero Knowledge Subtraction
We can also encrypt the operation a - b. This is the same as 

$$ g^{a}g{-b} = g^{a-b} $$

(rjp) i think he meant: $$ g^{a}g^{-b} = g^{a-b} $$


But we can't just stick a negative sign in front of the exponent

In [28]:
g = 57
n = 101
g ** -5 % n

1.6619797259514097e-09

The above is not an integer! However, python is smart enough to know what you are doing if you use `pow`. To test this, we expect

$$ g^{-a}g^{a} = 1     -->     g^{0} = 1 $$ 

because g^0 is 1, and `a - a = 0`.

In [29]:
a = 22
pow(g, a, n) * pow(g, -a, n) % n == 1       ## [ g^a (mod n)  * g^-a (mod n) ] mod (n)

True

So what magic is happening behind the scenes? The formula below is used to compute the modular inverse.

$$ a^{-1} = a^{n - 2} \pmod n $$                    

In [30]:
# using Fermat's Last theorem, which is the formula above

a_inv = a ** (n - 2) % n
a_inv * a % n == 1

True

## Multiplication by a constant
Multiplication by a constant is really just (modern zk proof algorithms do not use this technique, this is just for illustration purposes. 4x is really x + x + x + x. In encrypted form, this would be

$$ (g^{a})^{4} = g^{a} g^{a} g^{a} g^{a} $$

This allows us to "multiply by a constant" in a zero knowledge fashion.

In [31]:
a = 15
pow(g, a, n) * pow(g, a, n) * pow(g, a, n) * pow(g, a, n) % n == pow(g, a * 4, n)

True

Of course, it would be annoying to multiply like that if you have a big coefficient, so the following is better

In [32]:
# (g^a)^4 (mod n) == g^4a (mod n)
pow(pow(g, a, n), 4 , n) == pow(g, a * 4, n)

True

## I know the solution to a systems of equations

Now you should be able to prove to me that you know the solution to in a zero knowledge fashion

$$
2x + 8y = 7944\\
5x + 3y = 4764
$$

<font color='red'>**Assignment 4: Create a zk proof system where you can prove to a verifier (with an agreed upon g and n) that you know the solution to an agreed upon system of equation, without revealing the solution**</font>

## The field of rational numbers ℚ is homomorphic to the field ℤp

<font color='red'>**Assignment 5: Compute 53/192 + 61/511 (mod 1033) in python. Show your work. You are not required to implement euclid's algorithm.**</font>

In [33]:

frac1 = (53, 192)
frac2 = (61, 511)
modulus = 1033

# Find the inverse first: 53/192 (mod 1033)  --> [53 * 192^-1] (mod 1033)

# Numbers for which to find the modular inverse:
# 192: frac1[1]  
# 511: frac2[1]  

# modular inverses
mod_inverse_1 = pow(frac1[1], -1, modulus)    # 192^-1 
mod_inverse_2 = pow(frac2[1], -1, modulus)           # 511^-1

print("The modular inverse of {} modulo {} is: {}".format(frac1[1], modulus, mod_inverse_1))
print("The modular inverse of {} modulo {} is: {}".format(frac2[1], modulus, mod_inverse_2))

# 53/192 (mod 1033) 
result_a = pow(frac1[0] * mod_inverse_1, 1, modulus)
print("The result of 53/192 (mod 1033) is: {}".format(result_a))

# 61/511 (mod 1033) 
result_b = pow(frac2[0] * mod_inverse_2, 1, modulus)
print("The result of 61/511 (mod 1033) is: {}".format(result_a))

# 53/192 (mod 1033) + 61/511 (mod 1033) 
result = pow(result_a + result_b, 1, modulus)

print("The result of 53/192 + 61/511 (mod 1033) is: {}".format(result))


The modular inverse of 192 modulo 1033 is: 382
The modular inverse of 511 modulo 1033 is: 845
The result of 53/192 (mod 1033) is: 619
The result of 61/511 (mod 1033) is: 619
The result of 53/192 + 61/511 (mod 1033) is: 514
